In [1]:
import pandas as pd
from tables import *
import csv
import numpy as np
import pickle
import time
from scipy import sparse
from tqdm import tqdm_notebook as tqdm
#import multiprocessing as mp


def get_type(inqueue):
    try:
        for args_child in iter(inqueue.get,sentinel): 
            art_citant = args_child[0]
            art_cite = args_child[1]
            networks_citant = args_child[2]
            authors_citant = set(args_child[3])
            authors_cite = set(args_child[4])
            year_citant = args_child[5]
            start_time = args_child[6]
            out_path = args_child[7]
            sim = args_child[8]
            nb_citations = args_child[9]

            for aut_citant in authors_citant:
                if aut_citant in authors_cite:
                    type_cit = 0
                else:
                    if aut_citant in networks_citant:
                        network = networks_citant[aut_citant]
                        year_network = set(network[0,np.where(network[1,:]<=year_citant)[0]])
                        int_network = year_network.intersection(authors_cite)
                        if len(int_network) > 0:
                            type_cit = 1
                        else:
                            type_cit = 2
                    else:
                        type_cit = 2

                while True:
                    id_out = 0
                    try:
                        with open('{}output_{}.txt'.format(out_path,id_out),'a') as f:
                            #Citant,Cite,Sim,Typecit,AutCitant
                            f.write('{0},{1},{2:.4f},{3},{4}\n'.format(art_citant,art_cite,sim,type_cit,aut_citant))
                        break
                    except IOError:
                        id_out += 1

            elapsed_time = time.time() - start_time
            elapsed_time_h = np.round(elapsed_time/3600,2)
            while True:
                try:
                    with open('{}/progress.txt'.format(out_path), "r+") as f:
                        d = f.readlines()
                        line = d[0]
                        count = int(line.split('/')[0]) + 1
                        f.seek(0)
                        f.write('{}/{} lines, Time since start: {}, Queue size:{}\n'.format(count,nb_citations,elapsed_time_h,inqueue.qsize()))
                        f.truncate()
                        break
                except IOError:
                    time.sleep(0.01)
    except Exception as e:
         with open('{}error.txt'.format(out_path),'w') as f:
                f.write(str(e))

num_processes = mp.cpu_count()-1
sentinel = None
inqueue = mp.Queue()
jobs = []

for i in range(num_processes):
    p = mp.Process(target=get_type, args=(inqueue,))
    jobs.append(p)
    p.start()

In [2]:
path = "/mnt/disks/sdc/data/"


In [3]:
articles = {}
with open(path+'article.txt','r') as f:
    reader = csv.reader(f,delimiter='\t')
    next(reader)
    for line in reader:
        articles[int(line[0])] = int(line[2])

In [4]:
with open(path+'dict_cite.p','rb') as f:
    dict_cite = pickle.load(f)


In [5]:
with open(path+'dict_citant.p','rb') as f:
    dict_citant = pickle.load(f)

In [6]:
with open(path+'dict_cluster_art.p','rb') as f:
    dict_cluster_art = pickle.load(f)

In [7]:
with open(path+'dict_cluster_ID.p','rb') as f:
    dict_cluster_ID = pickle.load(f)

In [4]:
nb_aut_art = [len(dict_cluster_ID[x]) for x in dict_cluster_ID]

In [5]:
sum(nb_aut_art)

181362572

In [8]:
with open(path+'authors_disc.p','rb') as f:
    authors_disc_idx = pickle.load(f)
list_disciplines = list(authors_disc_idx.keys())
nb_disciplines = len(list_disciplines)

authors_info = {}
with open(path+'authors_info.csv','r') as f:
    reader = csv.reader(f,delimiter='\t')
    next(reader)
    for line in reader:
        authors_info[int(line[0])] = [line[1],int(line[2])]

In [ ]:
import time

In [ ]:
with open(path+'IDs_network.p','rb') as f:
    IDs_network = pickle.load(f)
authors_network = {}
count = 0
with open(path+'author_network.npy','rb') as f:
    while True:
        try:
            authors_network[IDs_network[count]] = np.load(f)
            count += 1
        except Exception:
            break
del IDs_network


### Type cit only

In [ ]:
authors = list(dict_cluster_ID.keys())
nb_authors = len(authors)
idx = np.arange(nb_authors)
np.random.shuffle(idx)

In [ ]:
f1 = open(path+'progress.txt','w')
count = 0
start_time = time.time()
with open(path+"type_ref_table.csv",'w') as f2:
    f2.write('ID_Citant,ID_Cite,Aut_Citant,Type_Cit\n')
    for i in range(nb_authors):
        count += 1
        author_ID = authors[idx[i]]
        arts_author = dict_cluster_ID[author_ID]
        if author_ID in authors_network:
            network = authors_network[author_ID]
            years_net = np.unique(network[1,:])
            dict_network = {}
            for year in years_net:
                dict_network[year] = set(network[0,np.where(network[1,:]<=year)[0]])
        else:
            dict_network = None
        for art_citant in arts_author:
            year_citant = articles[art_citant]
            if art_citant in dict_citant:
                arts_cite = dict_citant[art_citant]
                for art_cite in arts_cite:
                    if art_cite in dict_cluster_art:
                        authors_cite = set(dict_cluster_art[art_cite])
                        if author_ID in authors_cite:
                            type_cit = 0
                        else:
                            if dict_network:
                                id_years = years_net[np.where(years_net<=year_citant)[0]]
                                if len(id_years) > 0:
                                    year_network = np.max(id_years)
                                    network = dict_network[year_network]
                                    if len(authors_cite.intersection(network)) > 0:
                                        type_cit = 1
                                    else:
                                        type_cit = 2
                                else:
                                    type_cit = 2
                            else:
                                type_cit = 2
                        f2.write('{},{},{},{}\n'.format(int(art_citant),int(art_cite),int(author_ID),type_cit))
            
        if count % 1000 == 1 :
            elapsed_time = time.time() - start_time
            elapsed_time_h = np.round(elapsed_time/3600,2)
            time_per_art = elapsed_time/count
            time_left_h = np.round(time_per_art*(nb_authors-count)/3600,2)
            perc = np.round(count/nb_authors*100,3)
            f1.write('Progress: {} authors, {}%, Time since start: {}, Time left: {}\n'.format(count,perc,elapsed_time_h,time_left_h))    
            f1.flush()

### From refs table

In [7]:
h5file_sim = open_file(path + 'similarity_refs_only.h5', mode="r", title="similarity")
table_sim = h5file_sim.root.citations.sim
nb_citations = len(table_sim)


In [9]:
citations = range(nb_citations)
idx = np.arange(nb_citations)
np.random.shuffle(idx)
total_iterations = nb_citations

In [13]:
import shutil
import os

In [14]:
out_path = "/mnt/disks/sdc/data/type_cit/"

In [15]:
shutil.rmtree(out_path)

if not os.path.exists(out_path):
    os.makedirs(out_path)

with open('{}/progress.txt'.format(out_path),'w') as f:
    f.write('{}/{} lines, Time since start: {}\n'.format(0,nb_citations,0))

start_time = time.time()
for i in range(nb_citations):
    id_cit = idx[i]
    line = table_sim[id_cit]
    art_citant = line['Citant']
    art_cite = line['Cite']
    networks_citant = {}
    if art_citant in dict_cluster_art and art_cite in dict_cluster_art:
        authors_citant = dict_cluster_art[art_citant] #convert to set in parallel
        authors_cite = dict_cluster_art[art_cite] #convert to set in parallel
        year_citant = articles[art_citant] 
        for aut_citant in authors_citant:
            if aut_citant in authors_network:
                networks_citant[aut_citant] = authors_network[aut_citant]
                    
        args_child = [art_citant,art_cite,networks_citant,authors_citant,authors_cite,
                  year_citant,start_time,out_path,line['Sim'],nb_citations]
        inqueue.put(args_child)
    
for i in range(num_processes):
    # Send the sentinal to tell Simulation to end
    inqueue.put(sentinel)
    
for p in jobs:
    p.join()

for p in jobs:
    p.join()

In [12]:
class SimStats(IsDescription):
    Citant = UInt32Col()
    Cite = UInt32Col()
    Sim = Float16Col()
    TypeCit = UInt8Col()
    AutCitant = UInt32Col()

In [37]:
h5file_stats = open_file(path + 'sim_statistics.h5', mode="w", title="sim_stats")
group = h5file_stats.create_group("/", 'sim')
for j in tqdm(range(len(list_disciplines))):
    disc = list_disciplines[j]
    h5file_stats.create_table(group, disc, SimStats)


/home/User1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


A Jupyter Widget

/home/User1/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Biomedical Research'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/home/User1/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Clinical Medicine'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)
/home/User1/anaconda3/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'Earth and Space'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()``

In [36]:
h5file_stats.close()

### Serial

In [38]:
-

KeyboardInterrupt: 

In [34]:
def line_profiler():
    f = open(path+'progress.txt','w')
    start_time = time.time()
    count = 0
    for line in table_sim.iterrows(start=100000000,stop=100000200):
    #for i in range(100000000,100000200):
        #id_cit = idx[i]
        #line = table_sim[id_cit]
        art_citant = line['Citant']
        art_cite = line['Cite']
        if art_citant in dict_cluster_art and art_cite in dict_cluster_art:
            authors_citant = set(dict_cluster_art[art_citant])
            authors_cite = set(dict_cluster_art[art_cite])
            year_citant = articles[art_citant]
            for aut_citant in authors_citant:
                #info = authors_info[aut_citant]
                #disc = info[0]
                if aut_citant in authors_cite:
                    type_cit = 0
                else:
                    if aut_citant in authors_network:
                        network = authors_network[aut_citant]
                        year_network = set(network[0,np.where(network[1,:]<=year_citant)[0]])
                        int_network = year_network.intersection(authors_cite)
                        if len(int_network) > 0:
                            type_cit = 1
                        else:
                            type_cit = 2
                    else:
                        type_cit = 2
                table_stats = h5file_stats.get_node('/sim/{}'.format(disc))
                row = table_stats.row
                row['Citant'] = art_citant
                row['Cite'] = art_cite
                row['Sim'] = line['Sim']
                row['TypeCit'] = type_cit
                row['AutCitant'] = aut_citant
                row.append()
                table_stats.flush()
        if count % 20000 == 1 :
            elapsed_time = time.time() - start_time
            elapsed_time_h = np.round(elapsed_time/3600,2)
            time_per_art = elapsed_time/count
            time_left_h = np.round(time_per_art*(total_iterations-count)/3600,2)
            perc = np.round(count/total_iterations*100,3)
            f.write('Progress: {} articles, {}%, Time since start: {}, Time left: {}\n'.format(count,perc,elapsed_time_h,time_left_h))    
            f.flush()
        count += 1

In [17]:
%load_ext line_profiler

In [35]:
%lprun -f line_profiler line_profiler()

### Parallel

### Global

In [ ]:
def get_type(inqueue, id_out):
    for id_cit in iter(inqueue.get,sentinel):
        line = table_sim[id_cit]
        art_citant = line['Citant']
        art_cite = line['Cite']
        if art_citant in dict_cluster_art and art_cite in dict_cluster_art:
            authors_citant = set(dict_cluster_art[art_citant])
            authors_cite = set(dict_cluster_art[art_cite])
            year_citant = articles[art_citant]
            for aut_citant in authors_citant:
                info = authors_info[aut_citant]
                disc = info[0]
                if aut_citant in authors_cite:
                    type_cit = 0
                else:
                    if aut_citant in authors_network:
                        network = authors_network[aut_citant]
                        year_network = set(network[0,np.where(network[1,:]<=year_citant)[0]])
                        int_network = year_network.intersection(authors_cite)
                        if len(int_network) > 0:
                            type_cit = 1
                        else:
                            type_cit = 2
                    else:
                        type_cit = 2
        with open('{}output_{}.txt'.format(out_path,id_out),'a') as f:
            #Citant,Cite,Sim,Typecit,AutCitant
            f.write('{},{},{},{},{}\n'.format(art_citant,art_cite,line['Sim'],type_cit,aut_citant))


        elapsed_time = time.time() - start_time
        elapsed_time_h = np.round(elapsed_time/3600,2)
        while True:
            try:
                with open('{}/progress.txt'.format(out_path), "r+") as f:
                    d = f.readlines()
                    line = d[0]
                    count = int(line.split('/')[0]) + 1
                    f.seek(0)
                    fp.write('{}/{} lines, Time since start: {}\n'.format(count,nb_citations,elapsed_time_h))
                    f.truncate()
                    break
            except IOError:
                time.sleep(0.01)
    
    
    

In [ ]:
import shutil
import os

In [ ]:
out_path = "/mnt/disks/sdc/data/type_cit/"

In [ ]:
shutil.rmtree(out_path)

In [ ]:
if not os.path.exists(out_path):
    os.makedirs(out_path)
start_time = time.time()
with open('{}/progress.txt'.format(out_path),'w') as f:
    f.write('{}/{} lines, Time since start: {}\n'.format(0,nb_citations,0))

num_processes = 2#mp.cpu_count()-1
sentinel = None
inqueue = mp.Queue()
jobs = []

for i in range(num_processes):
    p = mp.Process(target=get_type, args=(inqueue,i))
    jobs.append(p)
    p.start()
for i in range(nb_citations):
    inqueue.put(idx[i])  
for i in range(num_processes):
    # Send the sentinal to tell Simulation to end
    inqueue.put(sentinel)
for p in jobs:
    p.join()


### Limited scope

In [ ]:
def get_type(inqueue):
    try:
        for args_child in iter(inqueue.get): 
            art_citant = args_child[0]
            art_cite = args_child[1]
            networks_citant = args_child[2]
            authors_citant = set(args_child[3])
            authors_cite = set(args_child[4])
            year_citant = args_child[5]
            start_time = args_child[6]
            out_path = args_child[7]
            sim = args_child[8]
            nb_citations = args_child[9]

            for aut_citant in authors_citant:
                if aut_citant in authors_cite:
                    type_cit = 0
                else:
                    if aut_citant in networks_citant:
                        network = networks_citant[aut_citant]
                        year_network = set(network[0,np.where(network[1,:]<=year_citant)[0]])
                        int_network = year_network.intersection(authors_cite)
                        if len(int_network) > 0:
                            type_cit = 1
                        else:
                            type_cit = 2
                    else:
                        type_cit = 2

                while True:
                    id_out = 0
                    try:
                        with open('{}output_{}.txt'.format(out_path,id_out),'a') as f:
                            #Citant,Cite,Sim,Typecit,AutCitant
                            f.write('{},{},{},{},{}\n'.format(art_citant,art_cite,sim,type_cit,aut_citant))
                        break
                    except IOError:
                        id_out += 1

            elapsed_time = time.time() - start_time
            elapsed_time_h = np.round(elapsed_time/3600,2)
            while True:
                try:
                    with open('{}/progress.txt'.format(out_path), "r+") as f:
                        d = f.readlines()
                        line = d[0]
                        count = int(line.split('/')[0]) + 1
                        f.seek(0)
                        f.write('{}/{} lines, Time since start: {}\n'.format(count,nb_citations,elapsed_time_h))
                        f.truncate()
                        break
                except IOError:
                    time.sleep(0.01)
    except Exception as e:
         with open('{}error.txt'.format(out_path),'w') as f:
                #Citant,Cite,Sim,Typecit,AutCitant
                f.write(str(e))


In [ ]:
import shutil
import os

In [ ]:
out_path = "/mnt/disks/sdc/data/type_cit/"

In [ ]:
shutil.rmtree(out_path)

In [ ]:
import multiprocessing as mp
mp.set_start_method("spawn")

In [ ]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

with open('{}/progress.txt'.format(out_path),'w') as f:
    f.write('{}/{} lines, Time since start: {}\n'.format(0,nb_citations,0))

#ctx = mp.get_context("spawn") # Use process spawning instead of fork
#P = ctx.Pool()

#P= mp.Pool()

num_processes = mp.cpu_count()-1
sentinel = None
inqueue = mp.Queue()
jobs = []

for i in range(num_processes):
    p = mp.Process(target=get_type, args=(inqueue,))
    jobs.append(p)
    p.start()
    
start_time = time.time()
for i in range(nb_citations):
    id_cit = idx[i]
    line = table_sim[id_cit]
    art_citant = line['Citant']
    art_cite = line['Cite']
    networks_citant = {}
    if art_citant in dict_cluster_art and art_cite in dict_cluster_art:
        authors_citant = dict_cluster_art[art_citant] #convert to set in parallel
        authors_cite = dict_cluster_art[art_cite] #convert to set in parallel
        year_citant = articles[art_citant] 
#         for aut_citant in authors_citant:
#             if aut_citant in authors_network:
#                     networks_citant = authors_network[aut_citant]
                    
        args_child = [art_citant,art_cite,networks_citant,authors_citant,authors_cite,
                  year_citant,start_time,out_path,line['Sim'],nb_citations]
        inqueue.put(args_child)
    
# for i in range(num_processes):
#     # Send the sentinal to tell Simulation to end
#     inqueue.put(sentinel)
    
for p in jobs:
    p.join()

In [ ]:
for p in jobs:
    p.terminate()

In [ ]:
inqueue.qsize()

In [ ]:
for i in range(num_processes):
    p = mp.Process(target=get_type, args=(inqueue,i,))
    jobs.append(p)
    p.start()

### With parallelpython

In [ ]:
def get_type(args_child):
    try:
        art_citant = args_child[0]
        art_cite = args_child[1]
        networks_citant = args_child[2]
        authors_citant = set(args_child[3])
        authors_cite = set(args_child[4])
        year_citant = args_child[5]
        start_time = args_child[6]
        out_path = args_child[7]
        sim = args_child[8]
        nb_citations = args_child[9]

        for aut_citant in authors_citant:
            if aut_citant in authors_cite:
                type_cit = 0
            else:
                if aut_citant in networks_citant:
                    network = networks_citant[aut_citant]
                    year_network = set(network[0,np.where(network[1,:]<=year_citant)[0]])
                    int_network = year_network.intersection(authors_cite)
                    if len(int_network) > 0:
                        type_cit = 1
                    else:
                        type_cit = 2
                else:
                    type_cit = 2

            while True:
                id_out = 0
                try:
                    with open('{}output_{}.txt'.format(out_path,id_out),'a') as f:
                        #Citant,Cite,Sim,Typecit,AutCitant
                        f.write('{},{},{},{},{}\n'.format(art_citant,art_cite,sim,type_cit,aut_citant))
                    break
                except IOError:
                    id_out += 1

        elapsed_time = time.time() - start_time
        elapsed_time_h = np.round(elapsed_time/3600,2)
        while True:
            try:
                with open('{}/progress.txt'.format(out_path), "r+") as f:
                    d = f.readlines()
                    line = d[0]
                    count = int(line.split('/')[0]) + 1
                    f.seek(0)
                    f.write('{}/{} lines, Time since start: {}\n'.format(count,nb_citations,elapsed_time_h))
                    f.truncate()
                    break
            except IOError:
                time.sleep(0.01)
    except Exception as e:
         with open('{}error.txt'.format(out_path),'w') as f:
                #Citant,Cite,Sim,Typecit,AutCitant
                f.write(str(e))


In [ ]:
import shutil
import os
out_path = "/mnt/disks/sdc/data/type_cit/"

In [ ]:
shutil.rmtree(out_path)

In [ ]:
if not os.path.exists(out_path):
    os.makedirs(out_path)
start_time = time.time()
with open('{}/progress.txt'.format(out_path),'w') as f:
    f.write('{}/{} lines, Time since start: {}\n'.format(0,nb_citations,0))

ppservers = ()
job_server = pp.Server(ppservers=ppservers)

    
for i in range(nb_citations):
    id_cit = idx[i]
    line = table_sim[id_cit]
    art_citant = line['Citant']
    art_cite = line['Cite']
    networks_citant = {}
    if art_citant in dict_cluster_art and art_cite in dict_cluster_art:
        authors_citant = dict_cluster_art[art_citant] #convert to set in parallel
        authors_cite = dict_cluster_art[art_cite] #convert to set in parallel
        year_citant = articles[art_citant] 
#         for aut_citant in authors_citant:
#             if aut_citant in authors_network:
#                     networks_citant = authors_network[aut_citant]
                    
        args_child = [art_citant,art_cite,networks_citant,authors_citant,authors_cite,
                      year_citant,start_time,out_path,line['Sim'],nb_citations]
        #Args: parallel function, args, functions to import (witout quotes), modules to import
        job_server.submit(get_type, (args_child,), (), ("time","numpy as np",))

In [ ]:
with open('{}/progress.txt'.format(out_path), "r+") as f:
    d = f.readlines()
    line = d[0]
    count = int(line.split('/')[0]) + 1
    f.seek(0)
    f.write('{}/{} lines, Time since start: {}\n'.format(count,nb_citations,0))
    f.truncate()